In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import statsmodels as sms
import statsmodels.api as sm
import math
import requests
import datetime

pd.set_option('display.max_rows',60000)
pd.set_option('display.max_columns',500)
pd.set_option('display.max_colwidth',200)
pd.options.display.float_format = '{:,.4f}'.format

# Portfolio Generation
## Access 19th Feb 2025

In [5]:
class SP500PortfolioBuilder:
    def __init__(self, low_needed, medium_needed, high_needed):
        self.low_needed = low_needed
        self.medium_needed = medium_needed
        self.high_needed = high_needed
        self.df = None
    
    def get_500_list(self):
        url = 'https://www.ssga.com/us/en/institutional/etfs/library-content/products/fund-data/etfs/us/holdings-daily-us-en-spy.xlsx'
        response = requests.get(url)
        
        if response.status_code == 200:
            with open('SPY_holdings.xlsx', 'wb') as file:
                file.write(response.content)
            
            df = pd.read_excel('SPY_holdings.xlsx', skiprows=4).dropna(how='any')
            df = df[~df['Name'].str.contains("US DOLLAR", na=False)].reset_index(drop=True)
            df['Ticker'] = df['Ticker'].replace({'BRK.B': 'BRK-B', 'BF.B': 'BF-B'})
            self.df = df[['Name', 'Ticker', 'Weight']].copy()
        else:
            raise Exception(f"Failed to download SPY holdings. Status code: {response.status_code}")
    
    def fetch_sectors(self):
        sectors = {}
        betas = {}
        
        tickers = self.df['Ticker'].tolist()
        
        for ticker in tickers:
            stock_info = yf.Ticker(ticker).info
            sectors[ticker] = stock_info.get('sector', 'Unknown')
            betas[ticker] = stock_info.get('beta', 0)
        
        self.df['Sector'] = self.df['Ticker'].map(sectors)
        self.df['Beta'] = self.df['Ticker'].map(betas)
        self.df = self.df[self.df['Beta'] != 0].reset_index(drop=True)
        self.df = self.df[~(self.df['Sector'] == 'Unknown')].reset_index(drop=True)
        
        # Handle GOOG and GOOGL weight merging
        googl_w = self.df[self.df['Ticker'] == 'GOOGL']['Weight'].values[0]
        goog_w = self.df[self.df['Ticker'] == 'GOOG']['Weight'].values[0]
        goog_new_w = googl_w + goog_w
        
        self.df.loc[self.df['Ticker'] == 'GOOG', 'Weight'] = goog_new_w
        self.df = self.df[~(self.df['Ticker'] == 'GOOGL')].reset_index(drop=True)
    
    @staticmethod
    def classify_beta(beta_value):
        if beta_value < 0.8:
            return 'Low'
        elif beta_value <= 1.2:
            return 'Medium'
        else:
            return 'High'
    
    def add_beta_risk(self):
        self.df['Beta_Risk'] = self.df['Beta'].apply(self.classify_beta)
    
    def build_portfolios_by_sector(self):
        portfolios = []
        grouped = self.df.groupby('Sector')
        
        for sector, group_df in grouped:
            group_sorted = group_df.sort_values(by='Weight', ascending=False)
            low_risk = group_sorted[group_sorted['Beta_Risk'] == 'Low'].sort_values(by='Weight', ascending=False)
            med_risk = group_sorted[group_sorted['Beta_Risk'] == 'Medium'].sort_values(by='Weight', ascending=False)
            high_risk = group_sorted[group_sorted['Beta_Risk'] == 'High'].sort_values(by='Weight', ascending=False)
            
            picks = [
                low_risk.head(self.low_needed),
                med_risk.head(self.medium_needed),
                high_risk.head(self.high_needed)
            ]
            
            sector_picks = pd.concat(picks, ignore_index=True)
            required_total = self.low_needed + self.medium_needed + self.high_needed
            
            if len(sector_picks) < required_total:
                shortage = required_total - len(sector_picks)
                selected_tickers = set(sector_picks['Ticker'])
                # Exclude already-selected tickers
                leftover = group_sorted.loc[~group_sorted['Ticker'].isin(selected_tickers)]
                # Take only the needed additional stocks
                filler = leftover.head(shortage)
                sector_picks = pd.concat([sector_picks, filler], ignore_index=True)
            
            sector_picks['Chosen_Sector'] = sector
            portfolios.append(sector_picks)
        
        return pd.concat(portfolios, ignore_index=True)
    
    def get_selected_portfolio(self, selection):
        self.get_500_list()
        self.fetch_sectors()
        self.add_beta_risk()
        
        mini_portfolios = self.build_portfolios_by_sector()
        mini_portfolios_select = mini_portfolios[mini_portfolios['Chosen_Sector'].isin(selection)].reset_index(drop=True)

        return mini_portfolios_select

In [6]:
selection = ['Communication Services', 'Consumer Cyclical', 'Energy', 
             'Financial Services', 'Healthcare', 'Technology'] 
portfolio_builder = SP500PortfolioBuilder(low_needed=2, medium_needed=2, high_needed=2)
mini_portfolios_select = portfolio_builder.get_selected_portfolio(selection)

In [8]:
# Normalize Weights within each Sector
sector_weights = mini_portfolios_select.groupby('Sector')['Weight'].transform(lambda x: x / x.sum())
mini_portfolios_select['Sector_Weight'] = sector_weights

In [11]:
mini_portfolios_select

,Name,Ticker,Weight,Sector,Beta,Beta_Risk,Chosen_Sector,Sector_Weight
0,AT+T INC,T,0.3595,Communication Services,0.7300,Low,Communication Services,0.0414
1,VERIZON COMMUNICATIONS INC,VZ,0.3333,Communication Services,0.4420,Low,Communication Services,0.0383
2,ALPHABET INC CL C,GOOG,3.7640,Communication Services,0.9910,Medium,Communication Services,0.4331
3,META PLATFORMS INC CLASS A,META,3.0022,Communication Services,1.2000,Medium,Communication Services,0.3454
4,NETFLIX INC,NFLX,0.8511,Communication Services,1.2740,High,Communication Services,0.0979
5,WALT DISNEY CO/THE,DIS,0.3814,Communication Services,1.4270,High,Communication Services,0.0439
6,MCDONALD S CORP,MCD,0.4200,Consumer Cyclical,0.7380,Low,Consumer Cyclical,0.0552
7,AUTOZONE INC,AZO,0.1099,Consumer Cyclical,0.7030,Low,Consumer Cyclical,0.0145
8,AMAZON.COM INC,AMZN,4.0775,Consumer Cyclical,1.1560,Medium,Consumer Cyclical,0.5361
9,HOME DEPOT INC,HD,0.7700,Consumer Cyclical,1.0580,Medium,Consumer Cyclical,0.1012


In [12]:
mini_portfolios_select.groupby('Sector').agg({'Beta':'mean'})

,Beta
Sector,
Communication Services,1.0107
Consumer Cyclical,1.2305
Energy,1.0567
Financial Services,0.9705
Healthcare,0.9740
Technology,1.0932


In [14]:
#mini_portfolios_select.to_csv('mini_portfolios_0219.csv',index=False)

In [ ]:
#https://finance.yahoo.com/calendar/earnings?symbol=CRM&offset=0&size=50